# Artificial Neural Network

## Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [2]:
tf.__version__

'2.18.0'

In [3]:
dataset_train = pd.read_csv("../train.csv")
X_train = dataset_train.drop(["Survived"], axis=1)
y_train = dataset_train["Survived"]
X_test = pd.read_csv("../test.csv")


In [4]:
numeric_features = ["Age", "SibSp", "Parch", "Fare"]
categorical_features = ["Sex", "Pclass", "Embarked"]


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)


In [6]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)


In [7]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


## Applying preprocessing

In [8]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


## Split the training data into training and validation sets

In [9]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_processed, y_train, test_size=0.2, random_state=42
)


## Build the ANN model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ann = Sequential(
    [
        Dense(64, activation="relu", input_shape=(X_train_processed.shape[1],)),
        Dropout(0.3),
        Dense(32, activation="relu"),
        Dropout(0.3),
        Dense(1, activation="sigmoid"),
    ]
)


C:\Users\GMT\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Compile the model

In [11]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


## Set up early stopping

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)


## Train the model

In [13]:
history = ann.fit(
    X_train_split,
    y_train_split,
    validation_data=(X_val_split, y_val_split),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=2
)


Epoch 1/100
23/23 - 2s - 87ms/step - accuracy: 0.6067 - loss: 0.6611 - val_accuracy: 0.7654 - val_loss: 0.5990
Epoch 2/100
23/23 - 0s - 8ms/step - accuracy: 0.7275 - loss: 0.5753 - val_accuracy: 0.7765 - val_loss: 0.5346
Epoch 3/100
23/23 - 0s - 8ms/step - accuracy: 0.7711 - loss: 0.5299 - val_accuracy: 0.8101 - val_loss: 0.4787
Epoch 4/100
23/23 - 0s - 7ms/step - accuracy: 0.7823 - loss: 0.4917 - val_accuracy: 0.8045 - val_loss: 0.4524
Epoch 5/100
23/23 - 0s - 8ms/step - accuracy: 0.7935 - loss: 0.4763 - val_accuracy: 0.8045 - val_loss: 0.4456
Epoch 6/100
23/23 - 0s - 7ms/step - accuracy: 0.7978 - loss: 0.4832 - val_accuracy: 0.8045 - val_loss: 0.4392
Epoch 7/100
23/23 - 0s - 7ms/step - accuracy: 0.8020 - loss: 0.4502 - val_accuracy: 0.8045 - val_loss: 0.4371
Epoch 8/100
23/23 - 0s - 7ms/step - accuracy: 0.8062 - loss: 0.4659 - val_accuracy: 0.8045 - val_loss: 0.4344
Epoch 9/100
23/23 - 0s - 7ms/step - accuracy: 0.7978 - loss: 0.4632 - val_accuracy: 0.8045 - val_loss: 0.4341
Epoch 10/

## Evaluate the model on the training set

In [14]:
train_loss, train_accuracy = ann.evaluate(X_train_split, y_train_split, verbose=0)
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")


Training Loss: 0.3795, Training Accuracy: 0.8427


## Evaluate the model on the validation set

In [15]:
val_loss, val_accuracy = ann.evaluate(X_val_split, y_val_split, verbose=0)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")


Validation Loss: 0.4248, Validation Accuracy: 0.8268


## Make predictions on the test set

In [16]:
y_pred = (ann.predict(X_test_processed) > 0.5).astype(int).flatten()


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


## Export to csv

In [17]:
results = pd.DataFrame({"PassengerId": X_test["PassengerId"], "Survived": y_pred})
results.to_csv("ann_submission.csv", index=False)
